In [1]:
import pandas as pd
import torch 

In [2]:
df = pd.read_csv('DWT_final.csv',index_col=0)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,411,412,413,414,415,416,417,418,419,Y
0,0.599773,-0.501864,0.134675,-0.533846,-0.405284,0.437332,0.217547,-0.438205,0.288973,-0.364478,...,0.243145,-0.925625,-0.932010,0.337195,2.819879,0.841384,-0.187117,1.595981,1.838580,0.0
1,-1.500665,1.491216,-0.964405,1.522549,-1.396584,1.597792,2.734664,0.133902,-0.314950,0.380534,...,-0.549256,1.664447,-0.562190,3.573534,2.521734,-2.466289,1.756804,-1.165767,-0.651851,1.0
2,0.498394,-0.720053,0.798192,-0.154170,0.180926,-0.020553,-0.316909,-0.234755,-0.504855,-0.254947,...,-0.494795,-1.109578,0.322932,-0.458588,-0.441847,0.951746,-0.071519,1.425615,1.130175,0.0
3,1.757567,0.816052,0.464469,-0.064927,-0.036567,0.017413,-0.914142,0.338207,0.126791,0.254954,...,-1.255586,-0.314968,2.279831,-0.389545,-0.581309,0.501069,-0.724476,0.126206,-1.391295,1.0
4,0.921110,-0.635559,0.617535,0.512857,0.150358,5.428818,0.042769,-0.339534,0.934680,5.900367,...,-1.370650,-0.441625,-0.898226,-0.390648,-0.572871,0.154312,-0.773098,-0.235034,-1.092907,0.0


In [4]:
X = df.drop('Y',axis=1).values

In [5]:
X = X.reshape(72,1,420)

In [6]:
X = torch.Tensor(X)

In [7]:
y = df['Y'].values

In [8]:
import torch
import torch.nn as nn
import snntorch as snn
from snntorch import surrogate
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

class SimpleCSNN(nn.Module):
    def __init__(self, input_channels, input_size, num_steps, beta=0.95):
        super(SimpleCSNN, self).__init__()
        self.num_steps = num_steps
        
        # Simplified convolutional layer
        self.conv1 = nn.Conv1d(input_channels, 16, kernel_size=5, stride=2)  # Reduced filters and kernel size
        
        # Calculate the output size of conv layer
        conv_out_size = (input_size - 5) // 2 + 1
        self.fc_input_size = 16 * conv_out_size

        # Leaky Integrate-and-Fire (LIF) layers
        self.sconv1 = snn.Leaky(beta=beta, spike_grad=surrogate.fast_sigmoid())

        # Reduced fully connected layer
        self.fc1 = nn.Linear(self.fc_input_size, 64)  # Reduced hidden units
        self.sfc1 = snn.Leaky(beta=beta, spike_grad=surrogate.fast_sigmoid())
        
        self.fc2 = nn.Linear(64, 1)  # Output layer for binary classification
        self.sfc2 = snn.Leaky(beta=beta, spike_grad=surrogate.fast_sigmoid())

    def forward(self, x):
        # x shape: (batch, channels, time)
        mem1, mem2 = self.sconv1.init_leaky(), self.sfc1.init_leaky()
        
        spk2_rec = []
        for _ in range(self.num_steps):
            cur1 = self.conv1(x)
            spk1, mem1 = self.sconv1(cur1, mem1)
            
            cur2 = self.fc1(spk1.view(spk1.size(0), -1))
            spk2, mem2 = self.sfc1(cur2, mem2)
            
            cur3 = self.fc2(spk2)
            spk3 = torch.sigmoid(cur3)  # Using sigmoid as the final activation

            spk2_rec.append(spk3)
        
        return torch.stack(spk2_rec, dim=0)

def train_model(model, optimizer, criterion, data, targets, num_epochs):
    model.train()
    for _ in range(num_epochs):
        optimizer.zero_grad()
        spk_rec = model(data)
        output = torch.mean(spk_rec, dim=0).squeeze()
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()
    return model

def evaluate_model(model, data, targets):
    model.eval()
    with torch.no_grad():
        spk_rec = model(data)
        output = torch.mean(spk_rec, dim=0).squeeze()
        predictions = (output > 0.5).float()
        return predictions.cpu().numpy(), targets.cpu().numpy()

# Hyperparameters
input_channels = 1  # Assuming single-channel EEG data
input_size = 420  
num_steps = 105  # Reduced the number of steps
num_epochs = 600 # Reduced the number of epochs
learning_rate = 2.5e-4
k_folds = 10  # Kept 10 folds for cross-validation
beta = 0.75
random_state=89

# Print hyperparameters and random state
print("Hyperparameters and Settings:")
print(f"Random state : {random_state}")
print(f"Input Channels: {input_channels}")
print(f"Input Size: {input_size}")
print(f"Number of Steps: {num_steps}")
print(f"Number of Epochs: {num_epochs}")
print(f"Learning Rate: {learning_rate}")
print(f"Number of Folds: {k_folds}")
print(f"Beta: {beta}")
print("\n")

# Convert data to PyTorch tensors (assuming X and y are your data and labels)
X = torch.FloatTensor(X)
y = torch.FloatTensor(y)

# Reshape X to (samples, channels, time)
X = X.view(X.size(0), input_channels, -1)

# Use StratifiedKFold
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=random_state)

fold_results = []
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"Fold {fold + 1}/{k_folds}")
    
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    model = SimpleCSNN(input_channels, input_size, num_steps, beta=beta)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.BCEWithLogitsLoss()

    model = train_model(model, optimizer, criterion, X_train, y_train, num_epochs)
    
    y_pred, y_true = evaluate_model(model, X_val, y_val)
    
    accuracy = accuracy_score(y_true, y_pred.round())
    precision = precision_score(y_true, y_pred.round())
    recall = recall_score(y_true, y_pred.round())
    f1 = f1_score(y_true, y_pred.round())
    
    fold_results.append({
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    })
    
    print(f"Fold {fold + 1} results: Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

# Calculate and print average results across all folds
avg_results = {metric: np.mean([fold[metric] for fold in fold_results]) for metric in fold_results[0]}
print("\nAverage results across all 10 folds:")
print(f"Accuracy: {avg_results['accuracy']:.4f}")
print(f"Precision: {avg_results['precision']:.4f}")
print(f"Recall: {avg_results['recall']:.4f}")
print(f"F1 Score: {avg_results['f1']:.4f}")

Hyperparameters and Settings:
Random state : 89
Input Channels: 1
Input Size: 420
Number of Steps: 105
Number of Epochs: 600
Learning Rate: 0.00025
Number of Folds: 10
Beta: 0.75


Fold 1/10
Fold 1 results: Accuracy: 0.8750, Precision: 0.8000, Recall: 1.0000, F1: 0.8889
Fold 2/10
Fold 2 results: Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000
Fold 3/10
Fold 3 results: Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000
Fold 4/10
Fold 4 results: Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000
Fold 5/10
Fold 5 results: Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000
Fold 6/10
Fold 6 results: Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000
Fold 7/10
Fold 7 results: Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000
Fold 8/10
Fold 8 results: Accuracy: 0.8571, Precision: 1.0000, Recall: 0.7500, F1: 0.8571
Fold 9/10
Fold 9 results: Accuracy: 0.8571, Precision: 0.8000, Recall: 1.0000, F1: 0.8889
Fold 10/1